# Lab 5: Simulating an N-qubit system

## Introduction: Building on Single Qubit Systems

Last week, you built a `SingleQubitSystem` class that simulated quantum states and operations on a single qubit. You implemented core methods to initialize quantum states, inspect them (measuring probabilities), and apply quantum gates like Hadamard and Pauli-Z. This week, we're taking a significant step forward: you'll generalize that single-qubit simulator to handle an **arbitrary number of qubits**.

In the previous assignment, you were provided with substantial skeleton code that outlined the class structure, method signatures, and helper functions. This scaffolding allowed you to focus on understanding the quantum mechanics and implementing the core logic. **In this assignment, you will be expected to do everything yourself.** You'll implement all required methods, and write comprehensive tests.


The jump from single-qubit to N-qubit systems requires a fundamental shift in how you think about quantum state representation.

In the `SingleQubitSystem`, you could hardcode quantum gate matrices—a 2×2 matrix for single-qubit gates is simple and manageable. Now, with N qubits, gate matrices scale exponentially: an operation on an N-qubit system requires a 2^N × 2^N matrix. Rather than hardcoding these, you'll need to **programmatically construct** gate matrices and tensor products that generalize to any number of qubits.

Beyond extending existing operations, in this lab you will also implement:

- **Multi-qubit gates**: You'll implement the SWAP and the CNOT gates.
- **Quantum oracles**: You'll implement two oracles—the **BernVaz oracle** and the **Archimedes oracle**.

# Task 0: Swapping bits

Before we dive into quantum mechanics, let's build intuition for how **SWAP** operations work at the bit level. Understanding classical bit swapping will help you reason about the quantum SWAP gate you'll implement later.

We've provided you with a helper function that converts integers to binary strings in **big-endian**:


In [2]:
def int_to_bin_string(n: int, number_of_bits: int) -> str:
    """
    Convert an integer to a binary string in big-endian format.

    Args:
        n: The integer to convert
        number_of_bits: The length of the output binary string

    Returns:
        A binary string of length number_of_bits in big-endian format

    """
    return format(n, f'0{number_of_bits}b')



A 4-bit system (called a **nibble**) can represent 16 different states (0 through 15). Your task is to print a truth table showing what happens when you swap the bits at **index 0** and **index 2**.

The truth table should have three columns:

1. **Original**: The original 4-bit string
2. **After Swap**: The 4-bit string after swapping bits at positions 0 and 2
3. **Decimal**: The decimal value of the resulting bit string

In [4]:
# Format your table like this:
original = after_swap = int_to_bin_string(3,4)
print(f'{original}\t{after_swap}')

## TODO: Print the truth table after swapping bits at positions 0 and 2

def swap_bits(bit_string: str, i: int, j: int) -> str:
    bits = list(bit_string)
    bits[i], bits[j] = bits[j], bits[i]
    return "".join(bits)


print("Original\tAfter Swap\tDecimal")

for n in range(16):
    original = int_to_bin_string(n, 4)
    swapped = swap_bits(original, 0, 2)
    decimal = int(swapped, 2)

    print(f"{original}\t{swapped}\t{decimal}")



0011	0011
Original	After Swap	Decimal
0000	0000	0
0001	0001	1
0010	1000	8
0011	1001	9
0100	0100	4
0101	0101	5
0110	1100	12
0111	1101	13
1000	0010	2
1001	0011	3
1010	1010	10
1011	1011	11
1100	0110	6
1101	0111	7
1110	1110	14
1111	1111	15



Now that you've seen how bit swapping works at the classical level, think about how this concept might extend to quantum systems.

Consider these questions as you reflect:

1. **State representation**: In your `SingleQubitSystem`, you represented a quantum state as a state vector—a list of amplitudes, one for each possible classical state. How might you use bit swapping to rearrange the elements of an N-qubit state vector?
2. **Permutation and basis states**: When you swap bits in a classical state, you're essentially permuting which basis state corresponds to which computational outcome. If bit swapping changes the binary representation, how does that change *which* element of the state vector you're modifying?
3. **Generalizing to multiple qubits**: In a 2-qubit system, there are 4 basis states ($|00\rangle$, $|01\rangle$, $|10\rangle$, $|11\rangle$). If you wanted to swap qubits 0 and 1, how would the indices of your state vector elements need to be rearranged? Can you see a pattern that would work for *any* pair of qubits in an N-qubit system?
4. **Efficiency**: Rather than constructing a full $2^N \times 2^N$ matrix, could you implement SWAP by directly rearranging elements of a state vector? What would be the computational advantage?



## `NQubitSystem` technical specs

`NQubitSystem` is a concrete implementation that inherits from the abstract `QubitSystem` base class. It generalizes single-qubit operations to support quantum systems with an arbitrary number of qubits.


| Function | Behavior |
| :-- | :-- |
| `__init__(num_qubits: int)` | Initializes an N-qubit system with `num_qubits` qubits. The initial state should be the computational basis state $\lvert 0 \ldots 0 \rangle$ (all qubits in state $\lvert 0 \rangle$). Raise a `ValueError` if `num_qubits` is less than 1. |
| `set_value(value: list[float])` | Sets the system's quantum state. Accept a list of floats representing the state vector amplitudes in **big-endian** basis order. The length of the input must equal $2^{\text{num_qubits}}$. Implementations should validate that the input represents a valid state (i.e., the sum of squared amplitudes equals 1.0, within numerical precision) and raise a `ValueError` if not. |
| `get_value_braket() -> str` | Returns a bra-ket style string representing the current state. This is primarily for readability/debugging, you won't be penalized for stylistic differences or formatting. |
| `get_value_vector() -> list[float]` | Returns the current state vector amplitudes in big-endian basis order. |
| `apply_not(i: int) -> None` | Applies the NOT gate (Pauli-$X$) to qubit at index $i$, updating the system state. Raise an `IndexError` if $i$ is not a valid qubit index. |
| `apply_h(i: int) -> None` | Applies the Hadamard gate ($H$) to qubit at index $i$, updating the system state. Raise an `IndexError` if $i$ is not a valid qubit index. |
| `apply_z(i: int) -> None` | Applies the Pauli-$Z$ gate to qubit at index $i$, updating the system state. Raise an `IndexError` if $i$ is not a valid qubit index. |
| `apply_cnot(control: int, target: int) -> None` | Applies a controlled-NOT gate with `control` as the control qubit and `target` as the target qubit, updating the system state. Raise an `IndexError` if either index is invalid, or if they are the same. |
| `apply_swap(i: int, j: int) -> None` | Swaps qubits at indices $i$ and $j$, updating the system state. Raise an `IndexError` if either index is invalid, or if they are the same. |
| `measure() -> str` | <p>Simulates a measurement of the state of the system and returns one of the possible values as a big-endian string of binary. For example, if the state is $\lvert 000 \rangle$, the result would always be `'000'`. If the state is $\frac{1}{\sqrt{2}} \lvert 000 \rangle + \frac{1}{\sqrt{2}} \lvert 101 \rangle$, measurement would return `'000'` or `'101'` with equal probability (50% each).<ul><li>Note: If a system is in a state of superposition before `measure()`, the act of measurement should collapse the superposition.</li><li>Note 2: The output should always have the same number of bits as the system does. For a 3-qubit system, outputs will be 3-character strings like `'000'`, `'101'`, etc.</li></ul></p> |


## Using Numpy


As with last week's lab, you are free to use `numpy` to handle mathematical operations. You may **not** use `qiskit` or **any** other library besides `numpy` as part of your solution. **Adding extra `import` statements _will_ crash the autograder.**

You can perform vector-matrix multiplication with `numpy` as follows:

In [5]:
import numpy as np

# Define a state vector (e.g., |01⟩ for a 2-qubit system)
state = np.array([0, 1, 0, 0])

# Define a gate (e.g., a 2-qubit gate, X on the second qubit)
gate = np.array([[0, 1, 0, 0],
                 [1, 0, 0, 0],
                 [0, 0, 0, 1],
                 [0, 0, 1, 0]])

# Apply the gate to the state using matrix-vector multiplication (Note: Numpy treats vectors as row-vectors)
new_state = gate @ state
print(new_state)
# Output: [1 0 0 0]

# Alternatively, using np.dot()
new_state = np.dot(gate, state)
print(new_state)

[1 0 0 0]
[1 0 0 0]


To compute the tensor product (also called the Kronecker product):

In [ ]:
import numpy as np

# Define two small matrices (e.g., single-qubit gates)
I = np.array([[1, 0],
              [0, 1]])

X = np.array([[0, 1],
              [1, 0]])

# Tensor product
result = np.kron(I, X)
print(result)

# Output:
#[[0 1 0 0]
# [1 0 0 0]
# [0 0 0 1]
# [0 0 1 0]]


## Task 1: Copy over your `QubitSystem` implementation from last week's assignment

In [ ]:
from abc import ABC, abstractmethod
import random
import numpy as np

## TODO: Paste QubitSystem here

## Task 2: Adapt your test suite from `SingleQubitSystem` for `NQubitSystem`
Just like last week, the autograder will expect the following tests to be present in your submission: `test_set_value`,`test_get_value_vector`,`test_apply_not`,`test_apply_h`,`test_apply_z`,`test_apply_cnot`,`test_apply_swap`,`test_measure`

In [ ]:
## TODO: Implement your test suite here



## Task 3: Implement `NQubitSystem`

In [ ]:
## TODO: Implement NQubitSystem Here



## Oracles:


In class we've discussed two quantum oracles: **BernVaz** and **Archimedes**. Notice how they share a common pattern:

1. **State Preparation**: Both oracles expect the system to be in a specific configuration. BernVaz, for example, expects all qubits must be in a superposition state (e.g., created by applying Hadamard gates to all qubits).
2. **Oracle Probing**: Once prepared, you apply the oracle itself, which transforms the superposed state according to the code(s) it encodes.
3. **Post-Processing**: After the oracle runs, the system requires additional quantum operations to extract useful information. In the case of BernVaz, this means applying Hadamard gates to all qubits again to collapse the superposition in a meaningful way.

Notice how both oracles follow this same high-level workflow, even though they encode different functions. This is precisely the kind of scenario where **object-oriented programming** shines.

## Interfaces: Capturing Shared Behavior

In OOP, an **interface** (called an abstract base class in python) defines a contract: "Any object that implements this interface must support these operations." Interfaces let you write code that works with *any* object following that contract, without needing to know the specific details of each implementation.

In our case, you might imagine defining an `Oracle` interface that specifies:

- An oracle must be able to prepare the system
- An oracle must be able to apply itself to a system
- An oracle must be able to post-process the result

Both `BernVazOracle` and `ArchimedesOracle` could implement this interface, each providing their own specific logic while adhering to the same structure. This way, future code could work with either oracle.

This pattern is powerful because it lets you **abstract away the differences** (number of qubits, specific gate sequences) while **capturing the similarities** (the three-step workflow). As quantum algorithms grow more complex, this kind of abstraction will help you manage complexity and extend your code without rewriting it.

# `Oracle` Abstract Base Class

Implement `Oracle` as an abstract base class (an `ABC` from the Python library `abc`) that defines the interface for quantum oracles (this means `Oracle` should consist entirely of `@abstractmethod`s). It is up to concrete oracle implementations to inherit from this class and provide concrete implementations of the three abstract methods.


| Function | Behavior |
| :-- | :-- |
| `__init__(codes: list[str])` | Initializes the oracle with a list of strings representing the 3-bit codes that define the oracle's behavior. Store these codes for use in the `probe()` method. |
| `pre_probe(system: NQubitSystem) -> None` | Prepares the quantum system into the state that the oracle expects. This typically involves creating an equal superposition across all qubits. Raise an `IndexError` if `system.num_qubits` is not equal to 4. |
| `probe(system: NQubitSystem) -> None` | Applies the oracle transformation to the system. The oracle uses the stored codes to determine how to transform each basis state. Raise an `IndexError` if `system.num_qubits` is not equal to 4. |
| `post_probe(system: NQubitSystem) -> None` | Performs post-processing on the system after the oracle has been applied. This typically involves applying additional gates (e.g., Hadamard gates) to extract meaningful information from the superposed state. Raise an `IndexError` if `system.num_qubits` is not equal to 4. |

## Task 4: Implement `Oracle` as an abstract base class

In [ ]:
## TODO: Oracle code below



## Task 5: Implement the `BernVaz` oracle as a child of `Oracle`
![bern vaz oracle](https://www.classes.cs.uchicago.edu/archive/2026/winter/22880-1/assigns/week5/figs/bernvazoraclealg.png)

**Note**: We will only test your code with code lists containing a single 3 bit code (e.g., `['011']`). It is up to you to decide how to handle the case of lists containing multiple codes; you may choose to ignore that case or throw an `IndexError` exception.

In [ ]:
## TODO: BernVaz code below



## Task 6: Implement the `Archimedes` oracle as a child of `Oracle`
![archimedes oracle](https://www.classes.cs.uchicago.edu/archive/2026/winter/22880-1/assigns/week5/figs/archoraclealg.png)


In [ ]:
## TODO: Archimedes code below



## Task 7: Test Your Oracles:

Finally, write tests to verify that your `Oracle` abstract base class and concrete oracle implementations work correctly. Implement the following:


| Test Name | Description |
| :-- | :-- |
| `test_oracle_is_abstract` | Verify that attempting to instantiate `Oracle` directly raises a `TypeError`. Since `Oracle` is an abstract base class, it should not be possible to create an instance of it without implementing all abstract methods. |
| `test_bernvaz_is_oracle` | Verify that `BernVazOracle` is a subclass of `Oracle` and can be instantiated with a valid list of 3-bit codes. |
| `test_archimedes_is_oracle` | Verify that `ArchimedesOracle` is a subclass of `Oracle` and can be instantiated with a valid list of 3-bit codes. |
| `test_bernvaz` | Create a 4-qubit `NQubitSystem`, prepare it using `BernVazOracle.pre_probe()`, apply the oracle using `BernVazOracle.probe()`, perform post-processing with `BernVazOracle.post_probe()`, then verify that the final state matches your expected result. Test with multiple different code lists to ensure correctness. |
| `test_archimedes` | Create a 4-qubit `NQubitSystem`, prepare it using `ArchimedesOracle.pre_probe()`, apply the oracle using `ArchimedesOracle.probe()`, perform post-processing with `ArchimedesOracle.post_probe()`, then verify that the final state matches your expected result. Test with multiple different code lists to ensure correctness. |




In [ ]:
## TODO: Oracle Tests



# Submission
Congratulations on completing the lab!
Make sure you:


1.   Test all of your functions by calling them at least once.
2.   Download your lab as a **python** `.py` script (NOT an `.ipynb` file):
      
      ```File -> Download -> Download .py```

3.   Rename the downloaded file to `Lab5Answers.py`.
4.   Upload the `Lab5Answers.py` file to Gradescope.
5.   Ensure the autograder runs successfully.